# Requirements
Requires an Nvidia GPU to run

Create a new anaconda environment and run the following commands to install the required libraries 
```
conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c conda-forge
conda install gensim
pip install torchdyn
pip install git+https://github.com/google-research/torchsde.git
```

# Citations
- Marcus, Mitchell P., Marcinkiewicz, Mary Ann & Santorini, Beatrice (1993). Building a Large Annotated Corpus of English: The Penn Treebank

```
@article{poli2020torchdyn,
  title={TorchDyn: A Neural Differential Equations Library},
  author={Poli, Michael and Massaroli, Stefano and Yamashita, Atsushi and Asama, Hajime and Park, Jinkyoo},
  journal={arXiv preprint arXiv:2009.09346},
  year={2020}
}
```

- GloVe

- GPT2 paper

- Huggingface for their implementation of transformers? Not sure if this has a paper


# To do
- Use BERT as a baseline and possibly an encoder
- Examine mini batch calculation
- Use LSTM function 
- Consider other variants of Neural ODE
- Implement and see results from my continuous language modelling idea

In [3]:
import torch
import torch.utils.data
import torchtext
import numpy as np
import gensim.downloader as api
from functools import reduce
from sklearn.metrics import *
from time import time

# Pre Processing
- Build the vocab
- Convert text corpus into padded word vector sequences

To do
- Use LSTM as baseline
    - Examine perplexity of model on validation set
- Implement Neural ODE

In [2]:
# load word embeddings
glove = api.load("glove-wiki-gigaword-300")

In [2]:
train, valid, test = torchtext.datasets.PennTreebank(split=('train', 'valid', 'test'))
train = list(train) # these are originally iterators, the data is so small we can just retrieve all of it at once
valid = list(valid)
test  = list(test)

In [3]:
# build the vocab
corpus = train + valid
vocab = {"<PAD>": 0}
index_vocab = {0 : "<PAD>"}
for sentence in corpus:
    for token in sentence.split(" ")[1:]:
        if token not in vocab:
            index = len(vocab)
            vocab[token] = index
            index_vocab[index] = token

# replace penn treebank end sentence token "\n" with glove's end sentence token "."
index = vocab["\n"]
vocab.pop("\n")         
vocab["."] = index
index_vocab[index] = "."

# view size
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)

Vocab size:  10001


In [4]:
# pad sentences and convert words to their glove vector to get input features
# convert to 1 hot vocab and shift 1 to the left to get output labels (converting to 1 hot takes too much memory, so just store indices and convert later)
# use left padding, as we want the hidden state at the end (right) to ignore the padding
# returns word_vector_dataset, labels
def preprocess(dataset, sequence_length, wv):
    embedding_size = wv["hello"].shape[0]
    processed = np.zeros((len(dataset), sequence_length, embedding_size))
    labels = np.zeros((len(dataset), sequence_length, 1))
    
    for i in range(len(dataset)):
        tokens = dataset[i].split(" ")[1:]
        
        # get the word vectors for all of the tokens, removing out of vocabulary (OOV) tokens
        tokens_np = np.zeros((len(tokens), embedding_size))
        labels_np = np.zeros((len(tokens), 1))
        j = 0
        for word in tokens:
            if word == "\n": word = "." # replace PennTreebank end sentence token '\n' with glove end sentence token "."
            if word not in wv: continue # ignore OOV tokens
            if j < sequence_length - 1: # only add sequence_length - 1 tokens at max
                # so that there is always a 0 vector at the start so the model learns most common starting words
                tokens_np[j, :] = wv[word]
            # we can look ahead to find the next word to set as the label for the last word
            if j < sequence_length:
                labels_np[j, :] = vocab[word]
            else: break
            j += 1
            
        tokens_np = tokens_np[:j-1, :]
        labels_np = labels_np[:j, :]
        
        # add this sentence to the overall dataset, with left padding of 0 vectors
        processed[i, sequence_length - tokens_np.shape[0]:, :] = tokens_np
        labels[i, sequence_length - labels_np.shape[0]:, :] = labels_np
    return processed, labels

In [6]:
sequence_length = 20
train_X, train_y = preprocess(train, sequence_length, glove)
valid_X, valid_y = preprocess(valid, sequence_length, glove)
test_X , test_y  = preprocess(test,  sequence_length, glove)

In [7]:
# unit test to check the labelling works
assert preprocess(["hello there how are you doing \n"], 20, glove)[1][0][-1] == 25, "Output: {}".format(preprocess(["hello there how are you doing \n"], 20, glove)[1][0])

# LSTM Baseline
Create a baseline RNN and evaluate it's perplexity

In [3]:
class LSTMModel(torch.nn.Module):
    def __init__(self, vocab_size, input_size=100, layer_size=100, dropout=0):
        super().__init__()
        self.LSTM = torch.nn.LSTM(input_size, layer_size, 1, bidirectional=False)
        self.dropout = torch.nn.Dropout(p=dropout)
        self.linear = torch.nn.Linear(layer_size, vocab_size)
        self.softmax = torch.nn.Softmax(dim=-1)
    
    def forward(self, x):
        # convert words to their vectors here
        sequence_outputs, hidden_state = self.LSTM(x)
        sequence_outputs = self.dropout(sequence_outputs)
        pred = self.linear(sequence_outputs)
        return pred
    
    # wrapper function that forward propagates, applies softmax and converts to numpy 
    def predict(self, x):
        preds = self.forward(x)
        preds = self.softmax(preds).detach().cpu().numpy()
        return preds

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def numpy_to_tensor(array):
    return torch.from_numpy(array).to(device).float()

In [5]:
model = LSTMModel(vocab_size, input_size=300, layer_size=300, dropout=0.1)
model.to(device)
model.eval()

NameError: name 'vocab_size' is not defined

In [12]:
%%time
# unit test to check that forward propagation works
data = numpy_to_tensor(train_X[:1000])
print(model.forward(data).shape)

torch.Size([1000, 20, 10001])
Wall time: 642 ms


In [13]:
# clear memory
del data
torch.cuda.empty_cache()

In [44]:
# Define functions to calculate perplexity for a single sentence: see the metric definition here https://web.stanford.edu/~jurafsky/slp3/3.pdf 
# We use teacher forcing (feeding the ground_truth label for sequence i to get pred for sequence i+1) to get the predictions
def perplexity(preds, ground_truth, epsilon=1e-30):
    probs = []
    for i in range(preds.shape[1]):
        probs.append(preds[0, i, int(ground_truth[i])])
    probs = np.array(probs)
    probs = np.power(1/(probs+epsilon), 1/probs.shape[0]) # normalise before taking the product, to prevent underflowing to 0
    return np.prod(probs)

# Calculate overall perplexity for a dataset
def average_perplexity(model, X, y):
    perplexities = [perplexity(model.predict(numpy_to_tensor(X[i:i+1])), y[i]) for i in range(X.shape[0])]
    return np.mean(perplexities)

In [15]:
# training the model
def train_model(model, train_X, train_y, epochs=10, learn_rate=0.01, weight_decay=0.001, minibatch_size=128, print_results=True):
    # Prepare data
    X = numpy_to_tensor(train_X)
    y = numpy_to_tensor(train_y).long()[:, :, 0]
    n_samples = X.shape[0]
    
    # Define loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate, weight_decay=weight_decay)

    # Ensure this runs on gpu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):      
        model.train() # set to train flag
        start_ts = time()
        
        # shuffle the data
        new_indices = torch.randperm(n_samples)
        X = X[new_indices, :, :] 
        y = y[new_indices, :]
        
        for batch_n in range(int(np.ceil(n_samples/minibatch_size))):
            # get the minibatch
            start_index = batch_n * minibatch_size
            end_index = min(start_index + minibatch_size, n_samples)
            batch_X = X[start_index: end_index, :, :]
            batch_y = y[start_index: end_index, :]
            
            # forward + backward + optimize
            optimizer.zero_grad()
            outputs = model(batch_X) 
            outputs = torch.swapaxes(outputs, 1, 2) # cross entropy expects a tensor of (n_samples, n_outputs, sequence_length)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
        # evaluate performance on part of the data (for memory reasons we take a subsample)
        if print_results:
            with torch.no_grad():
                model.eval()
                preds = np.argmax(X[:2000, :, :].detach().cpu().numpy(), axis=-1).flatten() # flatten the arrays so accuracy score works
                targets = y[:2000].detach().cpu().numpy().flatten()
                t_perplexity = average_perplexity(model, train_X[:2000], train_y[:2000])
                v_perplexity = average_perplexity(model, valid_X, valid_y)
                end_ts = time()
                print("Epoch {}, Minibatch loss: {:.2f}, Subsample Accuracy: {:.2f}, Train Perplexity: {:.2f}, Validation Perplexity: {:.2f}, Epoch Time: {:.2f} seconds".format(epoch, loss.item(),
                    accuracy_score(targets, preds), t_perplexity, v_perplexity, end_ts - start_ts))
    
    del X
    del y
    torch.cuda.empty_cache()
    if print_results:
        print('Finished Training')
    return model

In [23]:
model = train_model(model, train_X, train_y, epochs=200, learn_rate=0.001, minibatch_size=256, weight_decay=0.00001)

Epoch 0, Minibatch loss: 5.84, Subsample Accuracy: 0.19, Train Perplexity: 5476.68, Validation Perplexity: 5392.49
Epoch 1, Minibatch loss: 5.27, Subsample Accuracy: 0.19, Train Perplexity: 4591.04, Validation Perplexity: 4502.46
Epoch 2, Minibatch loss: 5.18, Subsample Accuracy: 0.18, Train Perplexity: 3781.74, Validation Perplexity: 3725.83
Epoch 3, Minibatch loss: 5.33, Subsample Accuracy: 0.18, Train Perplexity: 3172.56, Validation Perplexity: 3137.97
Epoch 4, Minibatch loss: 5.11, Subsample Accuracy: 0.18, Train Perplexity: 2724.83, Validation Perplexity: 2705.27
Epoch 5, Minibatch loss: 5.23, Subsample Accuracy: 0.19, Train Perplexity: 2389.16, Validation Perplexity: 2385.96
Epoch 6, Minibatch loss: 5.16, Subsample Accuracy: 0.18, Train Perplexity: 2150.90, Validation Perplexity: 2162.16
Epoch 7, Minibatch loss: 4.94, Subsample Accuracy: 0.19, Train Perplexity: 1951.61, Validation Perplexity: 1974.96
Epoch 8, Minibatch loss: 4.84, Subsample Accuracy: 0.19, Train Perplexity: 1790.

KeyboardInterrupt: 

In [18]:
torch.save(model.state_dict(), "LSTMBaseline.model") # save model

In [95]:
# load model, only run if the model hasn't already been trained
model.load_state_dict(torch.load("LSTMBaseline.model"))

<All keys matched successfully>

In [39]:
del model
torch.cuda.empty_cache()

### Hyper-Parameter tuning findings
- glove 300 dimension vectors are essential to not have a bias of 1000 perplexity on both train and validation
- 2 layers of LSTM also gives high bias, perhaps there is not enough train data
- Weight decay is essential in preventing Validation perplexity from skyrocketing
- Dropout of 0.1 combined with weight decay 0.00001 works (around 250 validation perplexity)
- Decreasing learning rate and increasing epochs has a minor benefit

### Examine Performance of the model
- Using both perplexity and qualitative evaluation

In [71]:
# view overall performance
model.eval()
data = numpy_to_tensor(train_X[:1000])
preds = model.predict(data)
#preds = torch.nn.Softmax(dim=-1)(preds).detach().cpu().numpy()

In [72]:
list(map(lambda x: index_vocab[x], np.argsort(preds[1, 11, :])[-10:]))

['probably',
 'help',
 'begin',
 'also',
 'take',
 'make',
 'continue',
 'the',
 'have',
 'be']

In [73]:
np.sort(preds[1][0])

array([5.1733707e-07, 5.4392586e-07, 5.5573935e-07, ..., 1.3860598e-02,
       3.9391726e-02, 7.3702824e-01], dtype=float32)

In [75]:
preds_int = np.argmax(preds, axis=-1)
for i in range(10):
    sentence = list(map(index_vocab.get, preds_int[i]))
    truth = list(map(lambda x: index_vocab[int(x)], train_y[i]))
    input_sentence = ['<PAD>'] + truth
    print("Predicted:", sentence)
    print("Input:    ",input_sentence)
    print()

Predicted: ['<PAD>', '<PAD>', 'securities', "'s", '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '.', '.', '<PAD>', '<PAD>', '<PAD>', 'the', '<PAD>', '<PAD>', '<PAD>', "'s", '<PAD>']
Input:     ['<PAD>', 'aer', 'banknote', 'berlitz', 'calloway', 'cluett', 'fromstein', 'gitano', 'guterman', 'ipo', 'kia', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'ssangyong', 'swapo', 'wachter']

Predicted: ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'the', '.', '.', 'be', 'the', 'company', "'s", 'a', 'new', 'director', 'of', '.']
Input:     ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'pierre', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', '.']

Predicted: ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'said', 'a', 'of', 'the', 'inc.', 'company', "'s", 'group', '.']
Input:     ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 

In [76]:
train[:10]

[' aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter \n',
 ' pierre <unk> N years old will join the board as a nonexecutive director nov. N \n',
 ' mr. <unk> is chairman of <unk> n.v. the dutch publishing group \n',
 ' rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate \n',
 ' a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported \n',
 ' the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said \n',
 ' <unk> inc. the unit of new york-based <unk> corp. that makes kent cigarettes stopped using <unk> in its <unk> cigarette filters in

In [77]:
pred1 = model.predict(numpy_to_tensor(valid_X[0:1]))
perplexity(pred1, valid_y[0])

80.610664

In [78]:
average_perplexity(model, train_X, train_y)

160.79262

In [79]:
average_perplexity(model, valid_X, valid_y)

245.91684

# GPT2 Baseline
Implement GPT2 as a language modelling baseline. GPT-3 is not publicly available and too large for practical purposes. BERT needs modification to work for language modelling, due to the fact that it is trained for bidirectional masked language modelling instead.

This section makes use of several tutorials for fine tuning, including:
- https://reyfarhan.com/posts/easy-gpt2-finetuning-huggingface/
- https://mccormickml.com/2019/07/22/BERT-fine-tuning/#4-train-our-classification-model
- https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel (the documentation)
- https://huggingface.co/transformers/custom_datasets.html

In [64]:
# load　GPT, BERT and support materials from huggingface
# requires pip install transformers
# if in jupyter notebook see here and you get an error mention ipython widgets see here: 
# https://stackoverflow.com/questions/53247985/tqdm-4-28-1-in-jupyter-notebook-intprogress-not-found-please-update-jupyter-an
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2LMHeadModel, top_k_top_p_filtering
from transformers import Trainer, TrainingArguments

In [65]:
train, valid, test = torchtext.datasets.PennTreebank(split=('train', 'valid', 'test'))
train = list(train) # these are originally iterators, the data is so small we can just retrieve all of it at once
valid = list(valid)
test  = list(test)

In [66]:
# Download the models
# Documentation for GPT: https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')

### Experimenting

In [67]:
# GPT2 example generation
text = f"Hugging Face is based in DUMBO, New York City, and"
tokens_tensor = gpt_tokenizer.encode(text, return_tensors="pt")

# Put everything on cuda
gpt_model.eval()
tokens_tensor = tokens_tensor.to('cuda')
gpt_model.to('cuda')

# Predict all tokens
generated = tokens_tensor
for i in range(20):
    next_token_logits = gpt_model(generated).logits[:, -1, :]
    # filter
    filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)
    # sample
    probs = torch.nn.functional.softmax(filtered_next_token_logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    generated = torch.cat([generated, next_token], dim=-1)

resulting_string = gpt_tokenizer.decode(generated.tolist()[0])
resulting_string

'Hugging Face is based in DUMBO, New York City, and is about a middle-aged man named John. When he wakes up in his dorm room to find'

In [68]:
tokens = gpt_tokenizer.encode(train[3])
print([gpt_tokenizer.decode([x]) for x in tokens])

[' rud', 'olph', ' <', 'unk', '>', ' N', ' years', ' old', ' and', ' former', ' chairman', ' of', ' consolidated', ' gold', ' fields', ' pl', 'c', ' was', ' named', ' a', ' nonex', 'ec', 'utive', ' director', ' of', ' this', ' b', 'rit', 'ish', ' industrial', ' conglomerate', ' ', '\n']


We can see that the gpt_tokenizer works differently to ours, splitting up names such as 'rudolph' into 'rud' and 'olph' and words such as nonexecutive and british. Hence our perplexity evaluation will have to be slightly different, using gpt_tokenizer to get the ground truth labels

### Data Processing

In [69]:
# Define a dataset class for fine-tuning, it's a generator so we don't have to store the entire dataset in memory
class GPT2Dataset(torch.utils.data.Dataset):
    def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=40):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        # Encode all the text, padding and truncuating it along with adding attention masks to get the sequence length the same across all samples
        for txt in txt_list:
            encodings_dict = tokenizer.encode_plus('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(encodings_dict['input_ids'])
            self.attn_masks.append(encodings_dict['attention_mask'])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # The tutorial use a dictionary format that also stores labels 
        return_dict = {"input_ids": torch.tensor(self.input_ids[idx]),
                       "attention_mask": torch.tensor(self.attn_masks[idx]), 
                       "labels": torch.tensor(self.input_ids[idx])} 
        return return_dict

In [70]:
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token # set the pad token
gpt_sequence_length = 40 # gpt splits up words into smaller tokens, so the sequence length should be longer
train_dataset = GPT2Dataset(train, gpt_tokenizer, max_length=gpt_sequence_length)
val_dataset = GPT2Dataset(valid, gpt_tokenizer, max_length=gpt_sequence_length)
train_dataset[1]

{'input_ids': tensor([   27,    91,  9688,  1659,  5239,    91,    29, 17748,   260,  1279,
          2954,    29,   399,   812,  1468,   481,  4654,   262,  3096,   355,
           257, 36196,   721,  8827,  3437,   645,    85,    13,   399,   220,
           198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([   27,    91,  9688,  1659,  5239,    91,    29, 17748,   260,  1279,
          2954,    29,   399,   812,  1468,   481,  4654,   262,  3096,   355,
           257, 36196,   721,  8827,  3437,   645,    85,    13,   399,   220,
           198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256])}

In [71]:
# get output by passing the ids and the attention mask
gpt_model(input_ids=train_dataset[1]['input_ids'].to(device), attention_mask=train_dataset[1]['attention_mask'].to(device), use_cache=False)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[-31.7761, -30.6999, -32.1561,  ..., -39.5131, -39.7474, -31.6402],
        [-63.3376, -60.9081, -61.4334,  ..., -72.3227, -72.0701, -62.4869],
        [-53.8055, -53.4089, -53.4207,  ..., -63.6040, -62.0605, -54.8142],
        ...,
        [-82.7951, -76.3731, -78.5615,  ..., -95.3905, -96.0068, -84.1115],
        [-82.7967, -76.3781, -78.5659,  ..., -95.3878, -96.0029, -84.1119],
        [-82.8332, -76.4163, -78.6067,  ..., -95.4225, -96.0345, -84.1454]],
       device='cuda:0', grad_fn=<MmBackward>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [72]:
gpt_tokenizer.decode(train_dataset[5]['input_ids'])

'<|startoftext|> the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said'

### Fine Tuning

Do fine tuning using the hugging face out of the box trainer https://huggingface.co/transformers/custom_datasets.html#fine-tuning-with-trainerfrom 

In [73]:
training_args = TrainingArguments(
    output_dir='gpt_finetuning',     # output directory
    num_train_epochs=1,              # total number of training epochs (1 is enough to get very low perplexity and perplexity increases at 2)
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='gpt_finetuning_logs',            # directory for storing logs
    logging_steps=100,
)

trainer = Trainer(
    model=gpt_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
100,5.079300
200,2.627500
300,2.437800
400,2.402000
500,2.320700
600,2.256900
700,2.208600
800,2.202500
900,2.203500
1000,2.159200


TrainOutput(global_step=2630, training_loss=2.287749967103675, metrics={'train_runtime': 265.3678, 'train_samples_per_second': 9.911, 'total_flos': 61761965506560.0, 'epoch': 1.0})

### Model Definition

In [74]:
# Build a model wrapper for gpt2 that uses the "past" variable and for language modelling
# TODO: Fine tuning - need to add padding for backprop in torch to work (just add '.' at the end for padding)
# TODO: add the options for beam search
class GPTModel(torch.nn.Module):
    def __init__(self, model=None, sequence_length=20):
        super().__init__()
        self.gpt = model.to(device)
        self.tokenizer = gpt_tokenizer
        self.sequence_length = sequence_length
        self.vocab_size = self.tokenizer.vocab_size
    
    # output the logits for the most likely next word at each position in the sentence
    # note input_dataset must be an element taken from a GPT2Dataset class (e.g. train_dataset[0])
    def forward(self, input_dataset):
        output = self.gpt.forward(input_ids = input_dataset['input_ids'].to(device), 
                                  attention_mask=input_dataset['attention_mask'].to(device),
                                  use_cache=False)
        return output["logits"]
    
    # take in a sentence and output the predictions as in forward, but as the most likely sentence not logits
    def forward_sentence(self, input_dataset):
        preds = self.forward(input_dataset)
        tokens = torch.argmax(preds, dim=-1)
        return self.tokenizer.decode(tokens)
    
    # generate a sentence by sampling the next word from the probability distribution
    # set limit to an integer to generate `limit` number of words instead of ending at a full stop
    def random_gen(self, x, limit=None):
        # initialize variables
        generated = self.tokenizer.encode_plus(x, return_tensors="pt")['input_ids'].to('cuda')
        next_token = [generated[0][-1]]
        past = None
        raw_output= None
        
        # generate until a "." is generated
        while (limit is None and self.tokenizer.decode(next_token[0]) not in [".", "?", "!"]) or (limit is not None and len(generated[0]) < limit):
            # get output of model, using past if available
            if past is None:
                raw_output = self.gpt(generated, past_key_values=past)
            else:
                raw_output = self.gpt(next_token, past_key_values=past)
            output, past = raw_output['logits'], raw_output['past_key_values']
            next_token_logits = output[:, -1, :]
            
            # sample a token from the top 50 most likely words
            filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0) # filter to the top 50 tokens
            probs = torch.nn.functional.softmax(filtered_next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            generated = torch.cat([generated, next_token], dim=-1)
            
        return self.tokenizer.decode(generated[0])
    
    # do beam_search to find the most likely sentence
    def beam_search(self, x, beam=5): 
        pass

In [75]:
modelgpt = GPTModel(model=gpt_model, sequence_length=gpt_sequence_length)
modelgpt.eval()
modelgpt.forward(train_dataset[0])

tensor([[-1.6970e+02, -1.6412e+02, -1.6493e+02,  ..., -2.0270e+02,
         -2.0474e+02, -1.6856e+02],
        [-2.6901e+00, -1.6066e+00,  4.1629e-01,  ..., -9.9957e+00,
         -1.0583e+01,  7.4749e-02],
        [-2.3482e+01, -2.3870e+01, -2.1071e+01,  ..., -4.0187e+01,
         -3.3782e+01, -2.0225e+01],
        ...,
        [-7.0412e+01, -7.0517e+01, -7.2410e+01,  ..., -7.6573e+01,
         -7.6298e+01, -6.6031e+01],
        [-6.7842e+01, -6.6881e+01, -6.5982e+01,  ..., -7.9229e+01,
         -7.9465e+01, -6.2473e+01],
        [-6.4527e+01, -6.3917e+01, -6.5069e+01,  ..., -7.2385e+01,
         -7.2153e+01, -6.0531e+01]], device='cuda:0', grad_fn=<MmBackward>)

View the next word output for a single example

### Evaluation

In [76]:
i = 18
input_t = [gpt_tokenizer.decode(t) for t in train_dataset[i]['input_ids']]
preds = modelgpt.forward(train_dataset[i])
tokens = torch.argmax(preds, dim=-1)
output_t = [gpt_tokenizer.decode(t) for t in tokens]
print("The format is:")
print("(Ground Truth word, Predicted next word),")
list(zip(input_t, output_t))

The format is:
(Ground Truth word, Predicted next word),


[('<', '|'),
 ('|', 'start'),
 ('start', 'of'),
 ('of', 'text'),
 ('text', '|'),
 ('|', '>'),
 ('>', ' the'),
 (' the', ' <'),
 (' total', ' value'),
 (' of', ' the'),
 (' N', ' N'),
 (' deaths', ' from'),
 (' from', ' <'),
 (' mal', 'ignant'),
 ('ignant', ' <'),
 (' <', 'unk'),
 ('unk', '>'),
 ('>', ' <'),
 (' lung', ' cancer'),
 (' cancer', ' in'),
 (' and', ' <'),
 (' <', 'unk'),
 ('unk', '>'),
 ('>', ' <'),
 (' was', ' N'),
 (' far', ' below'),
 (' higher', ' than'),
 (' than', ' the'),
 (' expected', ' '),
 (' the', ' study'),
 (' researchers', ' said'),
 (' said', ' '),
 (' ', '\n'),
 ('\n', '<|endoftext|>'),
 ('<|endoftext|>', '<|endoftext|>'),
 ('<|endoftext|>', '<|endoftext|>'),
 ('<|endoftext|>', '<|endoftext|>'),
 ('<|endoftext|>', '<|endoftext|>'),
 ('<|endoftext|>', '<|endoftext|>'),
 ('<|endoftext|>', '<|endoftext|>')]

View the ability to generate without teacher forcing using the random_gen() function

In [77]:
with torch.no_grad():
    result = modelgpt.random_gen("A new study says", limit=40)
result

'A new study says that the N to N share of the sales growth in the two sectors is far more than the number of sales that would be achieved in the N sector only when sales were concentrated in'

In [78]:
# if the mask is 0 at index i don't use the value at index i to calculate perplexity
def perplexity(preds, ground_truth, mask, epsilon=1e-30):
    probs = []
    for i in range(preds.shape[0]):
        if mask[i] != 0:
            probs.append(preds[i, int(ground_truth[i])])
    probs = np.array(probs)
    probs = np.power(1/(probs+epsilon), 1/probs.shape[0]) # normalise before taking the product, to prevent underflowing to 0
    return np.prod(probs).detach().cpu().numpy()

# Can optionally define n_samples=int to limit the number of samples used for perplexity evaluation
def average_perplexity_gpt(model, train, n_samples=None, print_results=False):
    perplexities = []
    n_samples = len(train) if n_samples is None else n_samples
    with torch.no_grad():
        for i in range(n_samples):
            # Compute perplexity for a single sample
            labels = train[i]['input_ids'][1:]
            mask = train[i]['attention_mask'][:-1]
            preds = model.forward(train[i])[:-1] # remove the last prediction as there is no ground truth 
            preds = torch.nn.functional.softmax(preds, dim=-1)
            perplexities.append(perplexity(preds, labels, mask))

            if i % 100 == 0 and print_results:
                print("Sentences analysed: {} Average perplexity: {}".format(i, np.mean(perplexities)))
    return np.mean(perplexities)

In [79]:
average_perplexity_gpt(modelgpt, train_dataset, n_samples=2000, print_results=True)

Sentences analysed: 0 Average perplexity: 1289.648193359375
Sentences analysed: 100 Average perplexity: 23.94414710998535
Sentences analysed: 200 Average perplexity: 18.348875045776367
Sentences analysed: 300 Average perplexity: 15.814547538757324
Sentences analysed: 400 Average perplexity: 14.741545677185059
Sentences analysed: 500 Average perplexity: 13.91878890991211
Sentences analysed: 600 Average perplexity: 13.654435157775879
Sentences analysed: 700 Average perplexity: 13.4156494140625
Sentences analysed: 800 Average perplexity: 13.6211519241333
Sentences analysed: 900 Average perplexity: 13.486576080322266
Sentences analysed: 1000 Average perplexity: 13.251569747924805
Sentences analysed: 1100 Average perplexity: 12.975820541381836
Sentences analysed: 1200 Average perplexity: 12.911108016967773
Sentences analysed: 1300 Average perplexity: 12.780496597290039
Sentences analysed: 1400 Average perplexity: 12.537127494812012
Sentences analysed: 1500 Average perplexity: 12.42807483673

12.501836

GPT2 Results
- No fine tuning: 706.16187 perplexity
- Fine tuning: 

# Neural ODE Model

In [15]:
import sys ; sys.path.append('../')
from torchdyn.models import *
from torchdyn.datasets import *
from torchdyn import *

In [16]:
# an Neural ODE that uses an LSTMCell as the derivative function
class ODELSTM(torch.nn.Module):
    def __init__(self, vocab_size, input_size=100, layer_size=100, dropout=0):
        super().__init__()
        self.LSTM = torch.nn.LSTM(input_size, layer_size, 1, bidirectional=False) # this encodes the sequence
        #self.fnode = torch.nn.LSTMCell(input_size, layer_size, 1, bidirectional=False)
        self.f = torch.nn.Sequential(
            torch.nn.Linear(layer_size, layer_size),
            torch.nn.ReLU(),
            torch.nn.Linear(layer_size, layer_size),
        )
        self.node = NeuralDE(self.f, sensitivity='adjoint', solver='dopri5').to(device)
        self.linear = torch.nn.Linear(layer_size, vocab_size)
        self.softmax = torch.nn.Softmax(dim=-1)
    
    def forward(self, x):
        # at the moment this feeds the entire sequence to LSTM and asks Neural ODE to reproduce it
        # TODO: switch to feeding half the sequence and asking NeuralODE to extrapolate
        sequence_outputs, hidden_state = self.LSTM(x)
        final_hidden = sequence_outputs[:, -1, :]

        # feed to neural ode
        timesteps = torch.linspace(0, sequence_length-1, sequence_length).to(device)
        sequence_outputs = self.node.trajectory(final_hidden, timesteps) # input is 128 final hidden states of dimension 300
        sequence_outputs = torch.swapaxes(sequence_outputs, 0, 1)        # output is output across 20 timesteps giving as (20, 128, 300) output, so swap the sequence and batch dimension
        
        # Get final output
        pred = self.linear(sequence_outputs)
        return pred
    
    # wrapper function that forward propagates, applies softmax and converts to numpy 
    def predict(self, x):
        preds = self.forward(x)
        preds = self.softmax(preds).detach().cpu().numpy()
        return preds

In [20]:
NODEmodel = ODELSTM(vocab_size, input_size=300, layer_size=300)

In [21]:
NODEmodel = train_model(NODEmodel, train_X, train_y, epochs=10, learn_rate=0.001, weight_decay=0)

Epoch 0, Minibatch loss: 5.62, Subsample Accuracy: 0.19, Train Perplexity: 992.81, Validation Perplexity: 949.61, Epoch Time: 334.68 seconds
Epoch 1, Minibatch loss: 5.38, Subsample Accuracy: 0.18, Train Perplexity: 825.13, Validation Perplexity: 815.57, Epoch Time: 438.83 seconds
Epoch 2, Minibatch loss: 5.46, Subsample Accuracy: 0.19, Train Perplexity: 718.59, Validation Perplexity: 734.18, Epoch Time: 517.79 seconds
Epoch 3, Minibatch loss: 5.50, Subsample Accuracy: 0.19, Train Perplexity: 699.71, Validation Perplexity: 762.84, Epoch Time: 598.56 seconds
Epoch 4, Minibatch loss: 5.32, Subsample Accuracy: 0.18, Train Perplexity: 602.00, Validation Perplexity: 666.84, Epoch Time: 600.94 seconds
Epoch 5, Minibatch loss: 5.17, Subsample Accuracy: 0.18, Train Perplexity: 601.95, Validation Perplexity: 663.96, Epoch Time: 599.20 seconds
Epoch 6, Minibatch loss: 5.11, Subsample Accuracy: 0.18, Train Perplexity: 1000.37, Validation Perplexity: 742.20, Epoch Time: 607.65 seconds
Epoch 7, Min

In [37]:
del NODEmodel
torch.cuda.empty_cache()